In [147]:
# In this script, we'll read in the keystore.json file and retrieve the private key, given a passphrase

In [148]:
from Crypto.Hash import keccak

In [149]:
from Crypto.Cipher import AES

In [150]:
from Crypto.Util import Counter

In [151]:
from Crypto.Protocol.KDF import scrypt

In [152]:
from eth_keys import keys

In [153]:
import json

In [154]:
import uuid

In [155]:
with open('keystore.json', 'r') as f:
    keystore = json.load(f)

In [156]:
ciphertext = bytearray.fromhex(keystore["crypto"]["ciphertext"])

In [157]:
iv = bytearray.fromhex(keystore["crypto"]["cipherparams"]["iv"])
number_of_rounds = keystore["crypto"]["kdfparams"]["n"]
r =  keystore["crypto"]["kdfparams"]["r"]
p =  keystore["crypto"]["kdfparams"]["p"]
dklen = keystore["crypto"]["kdfparams"]["dklen"]
salt = keystore["crypto"]["kdfparams"]["salt"]

In [158]:
passphrase = "Not secure"

In [172]:
original_encryption_key = scrypt(passphrase, bytearray.fromhex(salt), dklen, N=number_of_rounds, r=r, p=p)

In [173]:
print("Encryption key is {}".format(encryption_key.hex()))

Encryption key is 14256322ad2cf22311ec9c58e7904c69


In [174]:
iv_int = int.from_bytes(iv, "big")
print("Using block size of {} bytes".format(AES.block_size))
ctr = Counter.new(AES.block_size * 8, initial_value=iv_int)

Using block size of 16 bytes


In [175]:
# Only the first part of the original key is used for decryption
encryption_key = original_encryption_key[:16]

In [176]:
cipher_dec = AES.new(encryption_key, AES.MODE_CTR, counter=ctr)

In [177]:
decrypted_key = cipher_dec.decrypt(ciphertext)

In [178]:
print("Decrypted key is {}".format(decrypted_key.hex()))

Decrypted key is acaf3289d7b601cbd114fb36c4d29c85bbfd5e133f14cb355c3fd8d99367964f


In [179]:
# Upssss, you notice that your private key unlocked an empty wallet! Perhaps something's gone wrong during the decryption process?

In [180]:
# Let's double check if our decryption key is correct. We can check it against the mac address
mac_from_keyfile = bytearray.fromhex(keystore["crypto"]["mac"])

In [181]:
keccak_mac = keccak.new(digest_bits=256)
computed_mac = keccak_mac.update(original_encryption_key[16:]+ciphertext).digest()

In [182]:
print("The MAC we found was: \n{} \nwhile the computed MAC is: \n{}".format(mac_from_keyfile.hex(), computed_mac.hex()))
print("MACs match: {}".format(mac_from_keyfile==computed_mac))

The MAC we found was: 
7fd3326e8fd7325b7e95477bf7b04532bea423b0d2aa82952ecd8bb7ee368c85 
while the computed MAC is: 
7fd3326e8fd7325b7e95477bf7b04532bea423b0d2aa82952ecd8bb7ee368c85
MACs match: True


In [184]:
# So perhaps we provided the wrong passphrase?

In [185]:
passphrase = "Very secure"

In [186]:
# Re-run all the cells below the original declaration of passphrase and voilà